---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [4]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [5]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [10]:
def answer_one():
    s = df['Gold'].argmax()
    return s

In [11]:
answer_one()

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [14]:
def answer_two():
    s = abs(df['Gold']-df['Gold.1']).argmax()
    return s

In [15]:
answer_two()

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [25]:
def answer_three():
    df1 = df.copy()
    df1 = df[(df1['Gold']>0) & (df1['Gold.1']>0)]
    df1['diff']=abs(df1['Gold']-df1['Gold.1'])/(df1['Gold']+df1['Gold.1'])
    s = df1['diff'].argmax()
    return s

In [26]:
answer_three()

'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [27]:
df.shape

(146, 16)

In [28]:
def answer_four():
    df1 = df.copy()
    df1['Points'] = df1['Gold.2']*3 + df1['Silver.2']*2 + df1['Bronze.2']
    return df1['Points']

In [29]:
answer_four()

Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            923
Austria                              569
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              154
Belgium                              276
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             411
Burundi                                3
Cameroon                              12
Canada                               846
Chile                                 24
China                               1120
Colombia        

In [30]:
len(answer_four())

146

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [44]:
census_df = pd.read_csv('census.csv')
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [70]:
c_df = census_df.groupby(['STNAME']).size()
c_df

STNAME
Alabama                  68
Alaska                   30
Arizona                  16
Arkansas                 76
California               59
Colorado                 65
Connecticut               9
Delaware                  4
District of Columbia      2
Florida                  68
Georgia                 160
Hawaii                    6
Idaho                    45
Illinois                103
Indiana                  93
Iowa                    100
Kansas                  106
Kentucky                121
Louisiana                65
Maine                    17
Maryland                 25
Massachusetts            15
Michigan                 84
Minnesota                88
Mississippi              83
Missouri                116
Montana                  57
Nebraska                 94
Nevada                   18
New Hampshire            11
New Jersey               22
New Mexico               34
New York                 63
North Carolina          101
North Dakota             54
Ohio         

In [71]:
c_df.argmax()

'Texas'

In [74]:
def answer_five():
    df1 = census_df.copy()
    df1 = df1[df1['SUMLEV']==50]
    c_df = df1.groupby(['STNAME']).size()
    s = c_df.argmax()
    return s

In [75]:
answer_five()

'Texas'

### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [82]:
df1 = census_df.copy()
df1 = df1[df1['SUMLEV']==50]
feature = ['STNAME','CTYNAME','CENSUS2010POP']
df1 = df1[feature]
df1

,STNAME,CTYNAME,CENSUS2010POP
1,Alabama,Autauga County,54571
2,Alabama,Baldwin County,182265
3,Alabama,Barbour County,27457
4,Alabama,Bibb County,22915
5,Alabama,Blount County,57322
6,Alabama,Bullock County,10914
7,Alabama,Butler County,20947
8,Alabama,Calhoun County,118572
9,Alabama,Chambers County,34215
10,Alabama,Cherokee County,25989


In [87]:
largest3_df = df1.groupby('STNAME')['CENSUS2010POP'].nlargest(3)
largest3_df = largest3_df.reset_index()
largest3_df.head()

,STNAME,level_1,CENSUS2010POP
0,Alabama,37,658466
1,Alabama,49,412992
2,Alabama,45,334811
3,Alaska,71,291826
4,Alaska,76,97581


In [105]:
s = largest3_df.groupby('STNAME')['CENSUS2010POP'].sum().sort_values(ascending = False)
s.head(3)

STNAME
California    15924150
Texas          8269632
Illinois       6815061
Name: CENSUS2010POP, dtype: int64

In [108]:
def answer_six():
    df1 = census_df.copy()
    df1 = df1[df1['SUMLEV']==50]
    df1 = df1[['STNAME','CTYNAME','CENSUS2010POP']]
    largest3_df = df1.groupby('STNAME')['CENSUS2010POP'].nlargest(3)
    largest3_df = largest3_df.reset_index()
    largest3_df = largest3_df.groupby('STNAME')['CENSUS2010POP'].sum().sort_values(ascending = False).head(3)
    s = largest3_df.index.tolist()
    return s

In [109]:
answer_six()

['California', 'Texas', 'Illinois']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [111]:
df1 = census_df.copy()
df1 = df1[df1['SUMLEV']==50]
feature=['CTYNAME','POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']
df1 = df1[feature]
df1 = df1.set_index('CTYNAME')
df1.head()

,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015
CTYNAME,,,,,,
Autauga County,54660,55253,55175,55038,55290,55347
Baldwin County,183193,186659,190396,195126,199713,203709
Barbour County,27341,27226,27159,26973,26815,26489
Bibb County,22861,22733,22642,22512,22549,22583
Blount County,57373,57711,57776,57734,57658,57673


In [116]:
diff = df1.max(axis=1)-df1.min(axis=1)
diff.sort(ascending = False)
diff.head()

CTYNAME
Harris County         429841
Los Angeles County    344283
Maricopa County       342350
San Diego County      195135
Miami-Dade County     184946
dtype: int64

In [117]:
diff.argmax()

'Harris County'

In [121]:
def answer_seven():
    df1 = census_df.copy()
    df1 = df1[df1['SUMLEV']==50]
    feature = ['CTYNAME','POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']
    df1 = df1[feature]
    df1 = df1.set_index('CTYNAME')
    diff = df1.max(axis=1) - df1.min(axis=1)
    diff.sort(ascending = False)
    s = diff.argmax()
    return s

In [122]:
answer_seven()

'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [136]:
df1 = census_df.copy()
df1 = df1[df1['SUMLEV']==50]
df1.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411


In [138]:
df1 = df1[(df1['REGION']==1)|(df1['REGION']==2)]
df1

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
315,50,1,1,9,1,Connecticut,Fairfield County,916829,916850,919744,...,-0.228261,-3.030708,-2.070291,-6.633789,-7.385939,6.119788,2.687406,4.096896,0.441829,-0.338989
316,50,1,1,9,3,Connecticut,Hartford County,894014,894029,895303,...,-5.037297,-6.324143,-5.237664,-8.636074,-8.135161,-0.025666,-1.662136,-0.314193,-2.840433,-2.327200
317,50,1,1,9,5,Connecticut,Litchfield County,189927,189927,189773,...,-4.103317,-7.183847,-2.812353,-7.951141,-5.931948,-3.290047,-6.450584,-1.983618,-6.939080,-4.906204
318,50,1,1,9,7,Connecticut,Middlesex County,165676,165676,165628,...,1.410378,-6.716791,-2.617825,-4.987002,-6.526367,3.598271,-4.522077,-0.308335,-2.272328,-3.807554
319,50,1,1,9,9,Connecticut,New Haven County,862477,862474,863401,...,-5.240312,-5.579033,-6.802453,-7.571850,-8.434900,-1.074461,-1.736214,-2.755967,-2.767218,-3.620603
320,50,1,1,9,11,Connecticut,New London County,274055,274046,274067,...,-8.463706,-2.425106,-7.575122,-9.248635,-9.452705,-4.688659,2.432410,-3.302257,-4.571249,-4.381149
321,50,1,1,9,13,Connecticut,Tolland County,152691,152682,153208,...,-3.813541,-10.963827,-4.281160,-3.870513,-3.742364,-1.678219,-8.878601,-2.035198,-1.226432,-1.108849
322,50,1,1,9,15,Connecticut,Windham County,118428,118434,118593,...,-5.544046,-8.089355,-6.844197,-10.750119,-6.589461,-3.037834,-5.847013,-4.670358,-7.943144,-3.744596
609,50,2,3,17,1,Illinois,Adams County,67103,67103,67154,...,0.148846,-1.056728,-3.786524,-1.253600,-1.835396,0.416769,-0.803709,-3.518187,-0.955124,-1.536958
610,50,2,3,17,3,Illinois,Alexander County,8238,8238,8214,...,-24.274536,-41.452095,-66.340548,-21.245370,-42.513330,-24.028094,-41.324941,-66.072506,-20.965826,-42.225104


In [150]:
df1 = df1[df1['CTYNAME'].str.startswith('Washington')]
df1

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
896,50,2,4,19,183,Iowa,Washington County,21704,21704,21697,...,5.743693,2.468684,-0.364282,0.272171,1.849596,5.881542,2.560117,-0.273212,0.408256,1.984933
1419,50,2,4,27,163,Minnesota,Washington County,238136,238134,238995,...,2.555118,3.260957,2.285952,3.072543,0.678755,4.178076,4.907905,4.009584,5.040262,2.615204
2345,50,1,2,42,125,Pennsylvania,Washington County,207820,207820,207877,...,3.683135,3.260434,1.219337,2.113561,1.656917,4.062988,3.750219,1.742595,2.762039,2.305276
2355,50,1,1,44,9,Rhode Island,Washington County,126979,127094,127085,...,-4.669304,-2.659574,1.551904,1.400112,0.506035,-3.533527,-1.788880,2.462459,2.428442,1.533918
3163,50,2,3,55,131,Wisconsin,Washington County,131887,131885,131967,...,-0.794876,0.785279,-2.215465,1.601149,-0.434498,-0.431504,1.162817,-1.763330,2.104796,0.059931


In [151]:
df1 = df1[df1['POPESTIMATE2015']>df1['POPESTIMATE2014']]
df1

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
896,50,2,4,19,183,Iowa,Washington County,21704,21704,21697,...,5.743693,2.468684,-0.364282,0.272171,1.849596,5.881542,2.560117,-0.273212,0.408256,1.984933
1419,50,2,4,27,163,Minnesota,Washington County,238136,238134,238995,...,2.555118,3.260957,2.285952,3.072543,0.678755,4.178076,4.907905,4.009584,5.040262,2.615204
2345,50,1,2,42,125,Pennsylvania,Washington County,207820,207820,207877,...,3.683135,3.260434,1.219337,2.113561,1.656917,4.062988,3.750219,1.742595,2.762039,2.305276
2355,50,1,1,44,9,Rhode Island,Washington County,126979,127094,127085,...,-4.669304,-2.659574,1.551904,1.400112,0.506035,-3.533527,-1.788880,2.462459,2.428442,1.533918
3163,50,2,3,55,131,Wisconsin,Washington County,131887,131885,131967,...,-0.794876,0.785279,-2.215465,1.601149,-0.434498,-0.431504,1.162817,-1.763330,2.104796,0.059931


In [152]:
df1 = df1[['STNAME','CTYNAME']].sort_index()
df1

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County


In [154]:
def answer_eight():
    df1 = census_df.copy()
    df1 = df1[df1['SUMLEV']==50]
    df1 = df1[((df1['REGION']==1)|(df1['REGION']==2)) & (df1['CTYNAME'].str.startswith('Washington')) & (df1['POPESTIMATE2015']>df1['POPESTIMATE2014'])]
    s = df1[['STNAME','CTYNAME']].sort_index()
    return s

In [155]:
answer_eight()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
